## Thực hiện hợp đồng hoán đổi token trên mạng preprod

### Bước 0: Thiết kế hợp đồng trên Marlowe Playground

[Hợp đồng hoán đổi token giữa người mua và người bán](https://playground.marlowe-lang.org/#/importContract?marlowe-view=editor&contract=N4Ig7gFgpgdiBcBtUAXad6krByRp03wHsBrQ1M2AfRgEMBbKBEEAGhAGMBXAJ19icAntQDOQhgCNiAGxYgAvhxTEEoAA51eKIWpC9ZUaivIYQAOQDm3YgEsAQnTgKl%2BdHpMUSp2o2bwQAHEAeQAZABF2Lj4BGGExCWk5ALoAZgAGAHYAJjpMgBZOABN89M5UqAA2coBGAFZyyslsyoBOVrqAM1TOdKKa1JrWzIAOTvzOyUy6SUqoGuzJVrps9MVlVSJNbV0iAxkjTzMrG1sAWW46RVcCM04ZYlFmDk1d0E5iGFEUJxRqbcYLAAysRQjYYJYACpUZwcToGBjUOicD7cGAoPT7Q4wlgnOyOZyuV56D5fH7o-5aQEBEFgz6WULEABuUBkyOYrnhxERyNR6MxhmMOICePOl2uHE4dCeem2OgFByFplx1nxTnWIGInSVXiOviYLBCESiPH4ghE4ikshYaSyuQKxVK5SqtQaqSaLXaXR6fQGQ1G40m01m80Wy1WGts6OISJRxDRGL2gqOKtOBI1RSg6kethQohJn2%2Bv0pvGpIFp4KhOJcCgAuspbEx439SSgo5c258WPdHs98I2oM2PAOS2XsgA6SEQOwACScooJAGEINwhLBoaYAGp0Ygz4jqcL0iVcaX%2BDRaeVJxUpkWqsVXVxanUYSg%2BegGgImmLmhJW5KsVwoxUWM%2BUTUAsWfVM7AuB8OEzbNRFzfMiFJIsKQBBhgVBStGRZNlOA5Fx637JtuBbT42xgDtbC7AIexlBtSLAkijAwlgaknadbDnGBRRg5dV1gABBcIhO3Xd90PCFriAA)

### Bước 1: Tạo ví cho các bên liên quan

**Tạo thư mục tên là deploy-marlowe-contract để lưu các files liên quan**

In [ ]:
mkdir -p ~/notebooks/swap-token-contract

In [ ]:
cd ~/notebooks/swap-token-contract/

In [ ]:
pwd

**Tạo signing key và verification key cho các bên tham gia vào hợp đồng**

In [ ]:
cardano-cli address key-gen \
--verification-key-file nguoi-mua.vkey \
--signing-key-file nguoi-mua.skey

cardano-cli address key-gen \
--verification-key-file nguoi-ban.vkey \
--signing-key-file nguoi-ban.skey

ls

**Tạo địa chỉ ví từ verification key**

In [ ]:
cardano-cli address build \
--payment-verification-key-file nguoi-mua.vkey \
--testnet-magic 1 \
--out-file nguoi-mua.addr

cardano-cli address build \
--payment-verification-key-file nguoi-ban.vkey \
--testnet-magic 1 \
--out-file nguoi-ban.addr

ls

**Lưu lại đường dẫn đến các files signing key và địa chỉ ví của các bên tham gia vào các biến để sau này tiện sử dụng**

In [ ]:
BUYER_SKEY=nguoi-mua.skey
BUYER_ADDRESS=$(< nguoi-mua.addr)

SELLER_SKEY=nguoi-ban.skey
SELLER_ADDRESS=$(< nguoi-ban.addr)

In [ ]:
echo "Dia chi vi cua nguoi mua: $BUYER_ADDRESS"
echo "Dia chi vi cua nguoi ban: $SELLER_ADDRESS"

**Chuyển tADA vào ví của các bên, sau đó liệt kê các UTxO của từng địa chỉ ví xem đã nhận được tADA chưa**

_Chuyển 140 tADA vào ví của người mua, và 30 tADA và 10 native tokens vào ví của người bán_

In [ ]:
printf "\nCác UTXOs của người mua:\n"
cardano-cli query utxo --testnet-magic 1 --address $BUYER_ADDRESS

printf "\nCác UTXOs của người bán:\n"
cardano-cli query utxo --testnet-magic 1 --address $SELLER_ADDRESS

### Bước 2: Thiết lập các tham số dùng trong hợp đồng

In [ ]:
INITIAL_LOVELACE=2000000              # Khi triển khai hợp đồng Marlowe lên blockchain thì sẽ gửi tạm 2₳ vào hợp đồng
LOVELACE_AMOUNT=100000000             # Số lượng lovelace mà người mua cần chuyển vào hợp đồng
TOKEN_AMOUNT=10                       # Số lượng token mà người bán cần chuyển vào hợp đồng
MINIMUM_ADA=2000000                   # Số lovelace tối thiểu cần có cho mỗi UTxO có chứa native token là 2₳

BUYER_ROLE=NguoiMua                   # Đặt tên token của role buyer là NguoiMua
SELLER_ROLE=NguoiBan                  # Đặt tên token của role seller là NguoiBan

NOW=$(($(date -u +%s)*1000))          # Thời gian hiện tại tính theo POSIX milliseconds.

In [ ]:
echo "Thời gian hiện tại: $NOW (POSIX) => $(date -d @$(($NOW / 1000)) "+%H:%M:%S %d/%m/%Y UTC")"

### Bước 3: Tạo ra các role token và gửi vào địa chỉ ví của các bên tương ứng

In [ ]:
marlowe-cli --conway-era util mint --issuer $BUYER_ADDRESS:$BUYER_SKEY                   \
						--out-file /dev/null                                             \
						--submit 600                                                     \
						"$SELLER_ROLE":$SELLER_ADDRESS                                   \
						"$BUYER_ROLE":$BUYER_ADDRESS

**Kiểm tra xem token đã được mint và gửi đến đúng địa chỉ ví của các bên tương ứng chưa**

In [ ]:
for n in "$BUYER_ROLE" "$SELLER_ROLE"
do
  echo "$(echo -n $n | basenc --base16 | tr '[:upper:]' '[:lower:]') = $n"
done

In [ ]:
printf "\nCác UTXOs của người mua:\n"
cardano-cli query utxo --testnet-magic 1 --address $BUYER_ADDRESS

printf "\nCác UTXOs của người bán:\n"
cardano-cli query utxo --testnet-magic 1 --address $SELLER_ADDRESS

**Lưu lại UTxO của các bên để sử dụng cho bước tiếp theo**

In [ ]:
TX_0_BUYER_ADA_ONLY="52b2cdb4f40a51093f24d16164bcbe1d4f0bca23411ddbf3e4788f15a0940a18#0"
TX_0_BUYER_ROLE_TOKEN="52b2cdb4f40a51093f24d16164bcbe1d4f0bca23411ddbf3e4788f15a0940a18#2"

TX_0_SELLER_ADA_ONLY="be0d7749b041a88891fe85ebfa612e4b465a1a3f91331e815b3a05dc281e39c3#1"
TX_0_SELLER_ROLE_TOKEN="52b2cdb4f40a51093f24d16164bcbe1d4f0bca23411ddbf3e4788f15a0940a18#1"
TX_0_SELLER_NATIVE_TOKEN="be0d7749b041a88891fe85ebfa612e4b465a1a3f91331e815b3a05dc281e39c3#2"

**Lưu lại policy id và tên của các token**

In [ ]:
ROLES_CURRENCY=9745e9450d99579e8d5844da9c4282b002a46641c6da3be30417e2ae

BUYER_TOKEN="$ROLES_CURRENCY.$BUYER_ROLE"
SELLER_TOKEN="$ROLES_CURRENCY.$SELLER_ROLE"

echo "BUYER_TOKEN: $BUYER_TOKEN"
echo "SELLER_TOKEN: $SELLER_TOKEN"

### Bước 4: Triển khai và tương tác với hợp đồng trên blockchain

**Tạo 2 file JSON chứa nội dung và trạng thái của hợp đồng**

Thay vì sử dụng lệnh `marlowe-cli template`, người dùng có thể tạo hợp đồng Marlowe trên Marlowe Playground bằng Haskell, JavaScript và Blockly.
1. Thiết kế hợp đồng trên Marlowe Playground
2. Ấn vào nút "Send To Simulator"
3. Điền giá trị cho các tham số (nếu có)
4. Ấn vào nút "Download as JSON" để tải hợp đồng về máy
5. Đổi tên file JSON thành `tx-1.contract` (_không bắt buộc_)

Marlowe Playground không tạo ra file JSON lưu trạng thái ban đầu của hợp đồng, cho nên sau khi đã tải được file JSON có chứa nội dung hợp đồng, chúng ta cần tạo ra 1 file JSON nữa để lưu trạng thái ban đầu của hợp đồng. File đó sẽ chứa các thông tin về số dư của các tài khoản nội bộ, những lựa chọn đã thực hiện, những giá trị có thể chọn, và thời gian tối thiểu (POSIX) hợp đồng có thể bắt đầu.

Trong hợp đồng hoán đổi token này, chúng ta chỉ cần tạo 1 file JSON tên là `tx-1.state` với số dư tối thiểu `$INITIAL_LOVELACE` cho tài khoản nội bộ của vai trò sẽ triển khai hợp đồng lên blockchain. (Trong trường hợp này là vai trò `$BUYER_ROLE`)

In [ ]:
cat << EOI > tx-1.state
{
    "accounts": [
        [[{"role_token": "$BUYER_ROLE"}, {"currency_symbol": "", "token_name": ""}], $INITIAL_LOVELACE]
    ],
    "choices": [],
    "boundValues": [],
    "minTime": 1
}
EOI

#### Transaction 1: Người mua triển khai hợp đồng lên mạng blockchain

**Tạo file .marlowe chứa nội dung hợp đồng, trạng thái của nó, Plutus Validator và thông tin về network**

In [ ]:
marlowe-cli --conway-era run initialize --roles-currency "$ROLES_CURRENCY"        \
                           --permanently-without-staking                          \
                           --contract-file tx-1.contract                          \
                           --state-file    tx-1.state                             \
                           --out-file      tx-1.marlowe                           \
                           --print-stats

# permanently-without-staking => Reference Scripts (CIP-33)

**Dùng lệnh analyze để phân tích hợp đồng (file .marlowe) xem có thể chạy trên Cardano blockchain được không**

In [ ]:
marlowe-cli --conway-era run analyze --marlowe-file tx-1.marlowe

**Người mua sử dụng ADA của mình để triển khai hợp đồng**

In [ ]:
echo $BUYER_SKEY

In [ ]:
marlowe-cli --conway-era run execute --tx-in "$TX_0_BUYER_ADA_ONLY"      \
                        --change-address "$BUYER_ADDRESS"                \
                        --required-signer "$BUYER_SKEY"                  \
                        --marlowe-out-file tx-1.marlowe                  \
                        --out-file tx-1.raw                              \
                        --print-stats                                    \
                        --submit=600

**Lưu lại id của transaction để dùng cho bước tiếp theo**

In [ ]:
printf "\nCác UTXOs của người mua:\n"
cardano-cli query utxo --testnet-magic 1 --address $BUYER_ADDRESS

In [ ]:
TX_1=49f0a84f3ddc9c84330d97cd7308b51ac8a04defbb0fc5838756e54086dda288
TX_1_BUYER_ADA_ONLY="49f0a84f3ddc9c84330d97cd7308b51ac8a04defbb0fc5838756e54086dda288#0"

#### Transaction 2: Người mua gửi tiền vào tài khoản nội bộ của mình

**Tạo ra file .marlowe chứa thông tin về trạng thái tiếp theo của hợp đồng**

In [ ]:
marlowe-cli --conway-era run prepare --marlowe-file tx-1.marlowe           \
                        --deposit-account "$BUYER_ROLE"                    \
                        --deposit-party "$BUYER_ROLE"                      \
                        --deposit-amount "$LOVELACE_AMOUNT"                \
                        --invalid-before "$NOW"                            \
                        --invalid-hereafter "$((NOW+9*HOUR))"              \
                        --out-file tx-2.marlowe                            \
                        --print-stats

**Dùng lệnh analyze để phân tích hợp đồng (file .marlowe) xem có thể chạy trên Cardano blockchain được không**

In [ ]:
marlowe-cli --conway-era run analyze --marlowe-file tx-2.marlowe

**Thực hiện bước tiếp theo của hợp đồng**
_(Trạng thái hiện tại + Bước tiếp theo = Trạng thái tiếp theo)_

In [28]:
marlowe-cli --conway-era run execute --marlowe-in-file tx-1.marlowe           \
                        --tx-in-marlowe "$TX_1#1"                             \
                        --tx-in-collateral "$TX_1_BUYER_ADA_ONLY"             \
                        --tx-in "$TX_1_BUYER_ADA_ONLY"                        \
                        --tx-in "$TX_0_BUYER_ROLE_TOKEN"                      \
                        --required-signer "$BUYER_SKEY"                       \
                        --marlowe-out-file tx-2.marlowe                       \
                        --tx-out "$BUYER_ADDRESS+$MINIMUM_ADA+1 $BUYER_TOKEN" \
                        --change-address "$BUYER_ADDRESS"                     \
                        --out-file tx-2.raw                                   \
                        --print-stats                                         \
                        --submit=600

TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 1,ScriptErrorEvaluationFailed (CekError An error has occurred:
The machine terminated because of an error, either from a built-in function or from an explicit use of 'error'.) [])]


: 1

In [ ]:
marlowe-cli --conway-era run auto-execute --marlowe-in-file tx-1.marlowe      \
                        --tx-in-marlowe "$TX_1#1"                             \
                        --required-signer "$BUYER_SKEY"                       \
                        --marlowe-out-file tx-2.marlowe                       \
                        --change-address "$BUYER_ADDRESS"                     \
                        --out-file tx-2.raw                                   \
                        --print-stats                                         \
                        --submit=600

**Lưu lại id của transaction để dùng sau này**

In [ ]:
TX_2=3e94fb65f8eb6d7c1ac0508a1de0896da5f96167765f3a2892f216337d72ad51

#### Transaction 3: Người bán chuyển token vào tài khoản nội bộ của mình

**Tạo ra file .marlowe chứa thông tin về trạng thái tiếp theo của hợp đồng**

In [ ]:
marlowe-cli --conway-era run prepare --marlowe-file tx-2.marlowe           \
                        --deposit-account "$SELLER_ROLE"                   \
                        --deposit-party "$SELLER_ROLE"                     \
                        --deposit-amount "$TOKEN_AMOUNT"                   \
                        --invalid-before "$NOW"                            \
                        --invalid-hereafter "$((NOW+9*HOUR))"              \
                        --out-file tx-3.marlowe                            \
                        --print-stats

**Thực hiện bước tiếp theo của hợp đồng**
_(Trạng thái hiện tại + Bước tiếp theo = Trạng thái tiếp theo)_

In [ ]:
marlowe-cli --conway-era run execute --marlowe-in-file tx-2.marlowe             \
                        --tx-in-marlowe "$TX_2#1"                               \
                        --tx-in-collateral "$TX_0_SELLER_ADA_ONLY"              \
                        --tx-in "$TX_0_SELLER_ADA_ONLY"                         \
                        --tx-in "$TX_0_SELLER_ROLE_TOKEN"                       \
                        --required-signer "$SELLER_SKEY"                        \
                        --marlowe-out-file tx-3.marlowe                         \
                        --tx-out "$SELLER_ADDRESS+$MINIMUM_ADA+1 $SELLER_TOKEN" \
                        --change-address "$SELLER_ADDRESS"                      \
                        --out-file tx-3.raw                                     \
                        --print-stats                                           \
                        --submit=600

**Lưu lại id của transaction để dùng sau này**

In [ ]:
TX_3=085d4523f3df7e05495e0b7d660e05f87907060b33e72cc0ecafb604c935cbb4

#### Transaction 4: Người bán rút ADA khỏi hợp đồng

_Lệnh `marlowe-cli run withdraw` cho phép người bán rút ADA từ địa chỉ của Payout Script_

In [ ]:
marlowe-cli --conway-era run withdraw --marlowe-file tx-3.marlowe                     \
                           --role-name "$SELLER_ROLE"                                 \
                           --tx-in "$TX_4#0"                                          \
                           --tx-in "$TX_4#2"                                          \
                           --tx-in-collateral "$TX_4#0"                               \
                           --required-signer "$SELLER_SKEY"                           \
                           --tx-out "$SELLER_ADDRESS+$MINIMUM_ADA+1 $SELLER_TOKEN"    \
                           --change-address "$SELLER_ADDRESS"                         \
                           --out-file tx-4.raw                                        \
                           --print-stats                                              \
                           --submit=600

**Lưu lại id của transaction để dùng sau này**

In [ ]:
TX_4=d3cc866d1166ece150e1d430de6a493dd529081b9bfe5b21fff82baef122353b

#### Transaction 5: Người mua rút token khỏi hợp đồng

In [ ]:
marlowe-cli --conway-era run withdraw --marlowe-file tx-3.marlowe                    \
                         --role-name "$MEDIATOR_ROLE"                                \
                         --tx-in "$TX_5#0"                                           \
                         --tx-in "$TX_5#3"                                           \
                         --tx-in-collateral "$TX_5#0"                                \
                         --required-signer "$MEDIATOR_SKEY"                          \
                         --tx-out "$MEDIATOR_ADDRESS+$MINIMUM_ADA+1 $MEDIATOR_TOKEN" \
                         --change-address "$MEDIATOR_ADDRESS"                        \
                         --out-file tx-5.raw                                         \
                         --print-stats                                               \
                         --submit=600

In [ ]:
TX_5=df35e666e793610d73c8d8422d1c810c04463184eb9ca17da01f2b9847e9a7b2

#### Xem các transaction trên cardanoscan.io

In [ ]:
echo 'Transaction 1: https://preprod.cardanoscan.io/transaction/'"$TX_1"
echo 'Transaction 2: https://preprod.cardanoscan.io/transaction/'"$TX_2"
echo 'Transaction 3: https://preprod.cardanoscan.io/transaction/'"$TX_3"
echo 'Transaction 4: https://preprod.cardanoscan.io/transaction/'"$TX_4"
echo 'Transaction 5: https://preprod.cardanoscan.io/transaction/'"$TX_5"